# Imports

In [1]:
from modules.data import  data_reader, data_loader
from modules.models import lag_llama
from modules.experiment.tscv import get_tscv_results, get_summary, extract_metrics
from modules.visualization import graphs
from modules.fine_tuning import lag_llama_ft


In [2]:
"""
import os
print(os.getcwd())
"""

'\nimport os\nprint(os.getcwd())\n'

In [3]:
"""
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

# Format the date and time as a string
datetime_str = current_datetime.strftime("%Y-%d-%m %H:%M:%S")

# Return the date and time as a string
NOW = datetime_str
"""

'\nfrom datetime import datetime\n\n# Get the current date and time\ncurrent_datetime = datetime.now()\n\n# Format the date and time as a string\ndatetime_str = current_datetime.strftime("%Y-%d-%m %H:%M:%S")\n\n# Return the date and time as a string\nNOW = datetime_str\n'

# Parameters

In [4]:
# experiment parameters
#constraint: FOLDS x PREDICTION_LENGTH + TRAIN_SIZE !!must not!! be bigger than len(data)

PREDICTION_LENGTH = 1 # currentlz only works for PREDICTION_LENGTH > 1
TICKER = "AAPL"
FREQUENCY = "daily" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama", "autoregressor", "fine-tuned Llama"] # currentlz works onlz for these two
#FOLDS = 2 # for TSCV # reduced to two for testing purposes
CONTEXT_LENGTH = 245 # set to 245 for testing purposes
METRICS = ['r2', 'mse', 'mae', 'rmse', 'mda', "mape"]

# fine-tuning parameters
BATCH_SIZE = 10
MAX_EPOCHS = 2

# data parameters
FT_START_DATE = "2022-07-07"
START_DATE = "2023-07-07"
END_DATE = "2024-07-07"

# want to add
#TRAIN_PERIOD = # context lenghts. Should take a look into this
TRAIN_SIZE = CONTEXT_LENGTH



# Data

In [5]:
"""
data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY)
# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list
#if len(data) > 1:
data = data[0]

simple_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["simple"])
#if len(simple_data) > 1:
simple_data = simple_data[0]

train_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["train"])
train_data = train_data[0]

test_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["test"])
test_data = test_data[0]
"""

'\ndata = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY)\n# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list\n#if len(data) > 1:\ndata = data[0]\n\nsimple_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["simple"])\n#if len(simple_data) > 1:\nsimple_data = simple_data[0]\n\ntrain_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["train"])\ntrain_data = train_data[0]\n\ntest_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["test"])\ntest_data = test_data[0]\n'

In [6]:
DATA_CONFIG = {"ticker" : TICKER,
               "frequency" : FREQUENCY,
               "start" : START_DATE,
               "end" : END_DATE}

TRAIN_DATA_CONFIG = {"ticker" : TICKER,
                     "frequency" : FREQUENCY,
                     "start" : FT_START_DATE,
                     "end" : START_DATE}

In [7]:
data = data_loader.get_data(data_type=TYPE_OF_DATA, kwargs=DATA_CONFIG)
train_data = data_loader.get_data(data_type=TYPE_OF_DATA, kwargs=TRAIN_DATA_CONFIG)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
FOLDS = int((len(data) - TRAIN_SIZE) / PREDICTION_LENGTH) # this calculates max ammount of folds we can have given a set TRAIN_SIZE and PREDICTION_LENGTH

In [9]:
FOLDS

6

In [10]:
r, p = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY)

Fold 1/6 finished in: 6.28 seconds
Prediction from   2024-06-27 00:00:00   until   2024-06-27 00:00:00
----------------------


C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Fold 2/6 finished in: 6.50 seconds
Prediction from   2024-06-28 00:00:00   until   2024-06-28 00:00:00
----------------------


C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Fold 3/6 finished in: 5.93 seconds
Prediction from   2024-07-01 00:00:00   until   2024-07-01 00:00:00
----------------------


...

Fold 4/6 finished in: 6.51 seconds
Prediction from   2024-07-02 00:00:00   until   2024-07-02 00:00:00
----------------------


C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Fold 5/6 finished in: 5.79 seconds
Prediction from   2024-07-03 00:00:00   until   2024-07-03 00:00:00
----------------------


C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(5,0,0)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
                           ^^^^^^^^^^^^^^
  File "C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\pmdarima\arima\arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
             ^^^^^^^^^^
  File "C:\Users\topco\AppDat

Fold 6/6 finished in: 5.90 seconds
Prediction from   2024-07-05 00:00:00   until   2024-07-05 00:00:00
----------------------


In [11]:
r

,r2,mse,mae,rmse,mda,mape
arima,0.450407,14.482253,3.190176,3.805556,0.6,0.014594
lag_llama,-53.893603,1446.492957,37.615494,38.032788,0.6,0.171969
autoregressor,0.447312,14.563802,3.341670,3.816255,0.6,0.015282


In [12]:
p

,arima,lag_llama,autoregressor,actual
0,213.656341,174.820706,213.250000,214.100006
1,214.125477,182.567641,214.100006,210.619995
2,210.232611,182.634437,210.619995,216.750000
3,217.310340,183.517521,216.750000,220.270004
4,220.542241,175.931537,220.270004,221.550003
5,221.632904,184.465199,221.550003,226.339996


# Models

In [ ]:
# arima


In [ ]:
# lag llama

# Fine tuning an estimator

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=train_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\lightning\pytorch\trainer\configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | LagLlamaModel      | 2.4 M 
1 | augmentations | ApplyAugmentations | 0     
-----------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.797     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached -2.77185 (best -2.77185), saving model to 'c:\\Users\\topco\\Dokumenti\\MSc Banking and Digital Finance UCL\\Modules\\Dissertation\\MSc_dissertation\\lightning_logs\\version_1\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached -3.53485 (best -3.53485), saving model to 'c:\\Users\\topco\\Dokumenti\\MSc Banking and Digital Finance UCL\\Modules\\Dissertation\\MSc_dissertation\\lightning_logs\\version_1\\checkpoints\\epoch=1-step=100.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=2` reached.


# SKLearn TSCV

In [ ]:
r, p, a = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY, 
                           predictor=predictor)

C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Forecasting batches:   0%|          | 0/1 [00:00<?, ?it/s]

Fold 1/2 finished in: 9.75 seconds
Prediction from   2024-06-24   until   2024-06-28
----------------------


C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


Forecasting batches:   0%|          | 0/1 [00:00<?, ?it/s]

Fold 2/2 finished in: 10.72 seconds
Prediction from   2024-07-01   until   2024-07-08
----------------------


In [ ]:
#creating the summary for each model
s = [get_summary(r[i]) for i in range(len(r))]

In [ ]:
means_df, medians_df, stds_df = extract_metrics(s, MODELS)

# Visualisation

In [ ]:
graphs.standard_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.interactive_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.prediction_visualisation(MODELS, p, a)

LABEL the graph
label the axes with timestamps
show the history of the actual
table of comparison, not just graph